# Date Translation with Attention

In [12]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from utils import *
import matplotlib.pyplot as plt
%matplotlib inline

<a name='1-1'></a>
### Dataset

We will train the model on a dataset of 10,000 human readable dates and their equivalent, standardized, machine readable dates. Let's run the following cells to load the dataset and print some examples. 

In [39]:
fake = Faker()
Faker.seed(12345)
random.seed(12345)

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

LOCALES = ['en_US']

def load_date():
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS),  locale='en_US') # locale=random.choice(LOCALES))
        human_readable = human_readable.lower()
        human_readable = human_readable.replace(',','')
        machine_readable = dt.isoformat()
        
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt


def load_dataset(m):
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    Tx = 30
    

    for i in tqdm(range(m)):
        h, m, _ = load_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

In [40]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████████████████████████████| 10000/10000 [00:00<00:00, 23057.59it/s]


In [20]:
dataset[:10]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('08 jul 2008', '2008-07-08'),
 ('8 sep 1999', '1999-09-08'),
 ('thursday january 1 1981', '1981-01-01')]

In [21]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)


X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


In [22]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [23]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [41]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    
    concat = concatenator([a, s_prev])

    e = densor1(concat)

    energies = densor2(e)

    alphas = activator(energies)

    context = dotor([alphas,a])
    
    return context

In [42]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)

    for t in range(Ty):
    
        context = one_step_attention(a, s)
        
        s, _, c = post_activation_LSTM_cell(context,initial_state=[s, c])
        
        out = output_layer(s)
        
        outputs.append(out)
    
    model = Model(inputs=[X, s0, c0], outputs=outputs)
        
    return model

### Create Model

In [43]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

### get a summary of the model

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 37)]     0           []                               
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 30, 64)       17920       ['input_1[0][0]']                
                                                                                                  
 repeat_vector (RepeatVector)   (None, 30, 64)       0           ['s0[0][0]',                     
                                                                  'lstm[0][0]',               

                                                                  'bidirectional[0][0]',          
                                                                  'attention_weights[5][0]',      
                                                                  'bidirectional[0][0]',          
                                                                  'attention_weights[6][0]',      
                                                                  'bidirectional[0][0]',          
                                                                  'attention_weights[7][0]',      
                                                                  'bidirectional[0][0]',          
                                                                  'attention_weights[8][0]',      
                                                                  'bidirectional[0][0]',          
                                                                  'attention_weights[9][0]',      
          

### Compile model

In [44]:
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, decay=0.01) # Adam(...) 
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [45]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

### now fit the model and run it for 10 epoch.

In [35]:
model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10
  1/100 [..............................] - ETA: 6s - loss: 9.8319 - dense_2_loss: 0.2838 - dense_2_1_loss: 0.1840 - dense_2_2_loss: 1.1658 - dense_2_3_loss: 2.3313 - dense_2_4_loss: 0.0136 - dense_2_5_loss: 0.3761 - dense_2_6_loss: 2.0790 - dense_2_7_loss: 0.0170 - dense_2_8_loss: 1.1302 - dense_2_9_loss: 2.2510 - dense_2_accuracy: 0.9800 - dense_2_1_accuracy: 0.9500 - dense_2_2_accuracy: 0.4800 - dense_2_3_accuracy: 0.1000 - dense_2_4_accuracy: 1.0000 - dense_2_5_accuracy: 0.8800 - dense_2_6_accuracy: 0.1900 - dense_2_7_accuracy: 1.0000 - dense_2_8_accuracy: 0.5000 - dense_2_9_accuracy: 0.1600

2022-06-11 18:59:25.098989: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 44400000 exceeds 10% of free system memory.


100/100 [==============================] - 5s 54ms/step - loss: 9.2711 - dense_2_loss: 0.2031 - dense_2_1_loss: 0.1391 - dense_2_2_loss: 1.1088 - dense_2_3_loss: 2.2272 - dense_2_4_loss: 0.0119 - dense_2_5_loss: 0.2724 - dense_2_6_loss: 1.8390 - dense_2_7_loss: 0.0112 - dense_2_8_loss: 1.2023 - dense_2_9_loss: 2.2560 - dense_2_accuracy: 0.9579 - dense_2_1_accuracy: 0.9610 - dense_2_2_accuracy: 0.5139 - dense_2_3_accuracy: 0.1869 - dense_2_4_accuracy: 1.0000 - dense_2_5_accuracy: 0.9267 - dense_2_6_accuracy: 0.3000 - dense_2_7_accuracy: 1.0000 - dense_2_8_accuracy: 0.4428 - dense_2_9_accuracy: 0.1546
Epoch 2/10
100/100 [==============================] - 6s 57ms/step - loss: 8.1684 - dense_2_loss: 0.1404 - dense_2_1_loss: 0.1013 - dense_2_2_loss: 0.9829 - dense_2_3_loss: 2.0670 - dense_2_4_loss: 0.0104 - dense_2_5_loss: 0.1776 - dense_2_6_loss: 1.4417 - dense_2_7_loss: 0.0079 - dense_2_8_loss: 1.0958 - dense_2_9_loss: 2.1435 - dense_2_accuracy: 0.9672 - dense_2_1_accuracy: 0.9692 - dense

In [ ]:
### Test

In [46]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

source: 3 May 1979
output: 222---1111 

source: 5 April 09
output: 222---1111 

source: 21th of August 2016
output: 222----111 

source: Tue 10 Jul 2007
output: 222----111 

source: Saturday May 9 2018
output: 221----111 

source: March 3 2001
output: 221----111 

source: March 3rd 2001
output: 221----111 

source: 1 March 2001
output: 221----111 

